# MGMTMSA 408 – Operations Analytics – Spring 2024 - Final Exam

## 4 - Hotel revenue management

In [1]:
import numpy as np
import pandas as pd

In [2]:
hotel = pd.read_csv('hotel-probability.csv')
print(hotel.shape)
hotel.head()

(84, 4)


,r,din,dout,probability
0,Q,1,2,0.018519
1,Q,1,3,0.011111
2,Q,1,4,0.011111
3,Q,1,5,0.007407
4,Q,1,6,0.007407


In [3]:
sum(hotel['probability'])

0.7407407529999986

### Part 1: Understanding the data

In [4]:
# Probability that the hotel receives a reservation for a Queen room

hotel.loc[hotel['r'] == 'Q', 'probability'].sum()

0.3222222239999999

In [5]:
# Probability that the hotel receives a reservation that checks in on Thursday 9/14/2023 and checks out on Sunday 9/17/2023

hotel.loc[(hotel['din'] == 4) & (hotel['dout'] == 7), 'probability'].sum()

0.014814815

In [6]:
# Probability that there is no request

1 - hotel['probability'].sum()

0.259259247

In [7]:
# Expected value of the number of requests for a California King room, with check-in on Friday 9/15/2023 and check-out on Saturday 9/16/2023

T = 540

T * hotel.loc[(hotel['r'] == 'C') & (hotel['din'] == 5) & (hotel['dout'] == 6), 'probability'].sum()

5.99999994

### Part 2: Determining an optimal static allocation

In [8]:
n = hotel.shape[0] # number of request types
r = 3 # number of room types

# Number of rooms available for each room type across the 7 days:
B = np.array([[50, 50, 20]]* 7)

# Rooms used by requests of each type
rooms_Q = np.where(hotel['r'] == 'Q', 1, 0)
rooms_K = np.where(hotel['r'] == 'K', 1, 0)
rooms_C = np.where(hotel['r'] == 'C', 1, 0)
A = np.vstack((rooms_Q, rooms_K, rooms_C))

# Number of time periods
T = 540

# Expected demand
D = T * np.array(hotel['probability'])

# Revenue
revenue_room = {'Q': 200, 'K': 250, 'C': 300}

revenue = np.zeros(n)

for i in range(n):
    room = hotel.iloc[i, 0]
    base_rate = revenue_room[room]
    din = hotel.iloc[i, 1]
    dout = hotel.iloc[i, 2]
    total_days = dout - din
    num_days_additional = max(min(dout - 1, 6) - max(din - 1, 3), 0)
    num_days_base = total_days - num_days_additional
    revenue[i] = base_rate * num_days_base + base_rate * 1.15 * num_days_additional

In [9]:
# Formulate the LP:
from gurobipy import * 

# Create the model and the decision variables.
m = Model()

x = m.addVars(n, lb = 0, ub = D)

room_capacity_constrs = {}
for day in range(7):
    for ell in range(r):
        room_capacity_constrs[day, ell] = m.addConstr( sum(A[ell, i] * x[i] for i in range(n) if day+1>=hotel.iloc[i,1] and day+1<hotel.iloc[i,2]) <= B[day, ell])

# Specify the objective
m.setObjective( sum(revenue[i] * x[i] for i in range(n)), GRB.MAXIMIZE)

# Solve 
m.update()
m.optimize()

# Save the LP objective
LP_obj = m.objval

print(f'The optimal revenue is ${LP_obj}.')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-17
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 84 columns and 252 nonzeros
Model fingerprint: 0x995dd249
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 2e+03]
  Bounds range     [2e+00, 4e+01]
  RHS range        [2e+01, 5e+01]
Presolve removed 9 rows and 16 columns
Presolve time: 0.00s
Presolved: 12 rows, 68 columns, 159 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1148000e+05   1.135000e+02   0.000000e+00      0s
Extra simplex iterations after uncrush: 1
      16    1.7214000e+05   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.721400018e+05
The optimal revenue is $172140.0017788.


In [10]:
# Static allocation
requests_accepted = np.array([x[i].x for i in range(n)])

# Indices of top 5 request types
ind = np.argpartition(requests_accepted, -5)[-5:]
ind = np.flipud(ind[np.argsort(requests_accepted[ind])])
print(ind)

# Number of requests accepted
print(requests_accepted[np.flipud(ind[np.argsort(requests_accepted[ind])])])

# Details of top 5 request types accepted
hotel.iloc[ind]

[35 41 46 36  0]
[19.99999814 15.99999782 13.9999982  10.00000026 10.00000026]


,r,din,dout,probability
35,K,2,3,0.037037
41,K,3,4,0.037037
46,K,4,5,0.029630
36,K,2,4,0.018519
0,Q,1,2,0.018519


In [11]:
# Optimal values of the dual variables for the resource constraints
dual_optimal = [[room_capacity_constrs[day, ell].pi for ell in range(r)] for day in range(7)]
dual_optimal

[[0.0, 0.0, 0.0],
 [200.0, 250.0, 300.0],
 [200.0, 250.0, 300.0],
 [230.0, 287.5, 345.0],
 [229.99999999999997, 0.0, 345.0],
 [0.0, 0.0, 345.0],
 [0.0, 0.0, 0.0]]

In [12]:
# Predicted change in the revenue on converting 10 King’s room into Queen’s rooms
10 * sum([dual_optimal[day][0] - dual_optimal[day][1] for day in range(len(dual_optimal))])

724.9999999999998

### Part 3: Determining an optimal dynamic allocation policy

In [13]:
# Generating 100 random sequences
nSimulations = 100
np.random.seed(50)

probability_aug = np.zeros(n+1)
probability_aug[0:n] = np.array(hotel['probability']) # First nItineraries elements are the same as probability
probability_aug[n] = 1 - sum(np.array(hotel['probability'])) # Last element is one minus the rest.

arrival_sequences = np.zeros((nSimulations, T))

for s in range(nSimulations):
    arrival_sequences[s] = np.random.choice(range(n+1), T, p=probability_aug)

arrival_sequences

array([[46., 14., 17., ..., 36., 29., 41.],
       [84., 84., 18., ..., 84.,  7., 64.],
       [84.,  9., 36., ...,  3., 25., 51.],
       ...,
       [22., 13., 47., ..., 42., 84., 84.],
       [ 7., 69., 84., ..., 22., 79., 41.],
       [23., 75., 75., ..., 50., 84., 59.]])

In [14]:
# Mapping from request type to room type
req_to_rooms = list(np.where(hotel['r'] == 'Q', 0, np.where(hotel['r'] == 'K', 1, 2)))
req_to_rooms

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [15]:
# Average revenue for first come first serve policy
results_myopic_revenue = np.zeros(nSimulations)

for s in range(nSimulations):
    total_revenue = 0.0
    b = B.copy()
    
    for t in range(T):
        
        i = int(arrival_sequences[s, t])
        
        if (i < n):
            days = [day for day in range(7) if day+1>=hotel.iloc[i,1] and day+1<hotel.iloc[i,2]]
            if (b[days, req_to_rooms[i]] > 0).all():
                b[days, req_to_rooms[i]] -= 1
                total_revenue += revenue[i]
    
    results_myopic_revenue[s] = total_revenue
    
mean_myopic_revenue = results_myopic_revenue.mean()

print(f"Average revenue for first come first serve policy over the 100 random requests sequences is ${mean_myopic_revenue}")

Average revenue for first come first serve policy over the 100 random requests sequences is $147472.025


In [16]:
probability = np.array(hotel['probability'])
probability

array([0.01851852, 0.01111111, 0.01111111, 0.00740741, 0.00740741,
       0.0037037 , 0.0037037 , 0.01111111, 0.03703704, 0.01851852,
       0.00740741, 0.00740741, 0.0037037 , 0.07407407, 0.01481482,
       0.00740741, 0.00740741, 0.0037037 , 0.0037037 , 0.01481482,
       0.0037037 , 0.0037037 , 0.02222222, 0.0037037 , 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.01481482, 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 ,
       0.03703704, 0.01851852, 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.03703704, 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.02962963, 0.01481482, 0.00740741, 0.0037037 ,
       0.01481482, 0.00740741, 0.00740741, 0.0037037 , 0.0037037 ,
       0.0037037 , 0.00740741, 0.00740741, 0.00740741, 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.01111111, 0.01851852,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.01851852,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.00370

In [17]:
# Define a function to repeatedly solve
# the LP at different periods and with different numbers of remaining seats. 
def bpc(b,t):
    for day in range(7):
        for ell in range(r):
            room_capacity_constrs[day, ell].rhs = b[day, ell]
    
    for i in range(n):
        x[i].ub = (T - t)*probability[i]
    
    m.update()
    m.optimize()
    
    dual_val = [[room_capacity_constrs[day, ell].pi for ell in range(r)] for day in range(7)]
    
    return dual_val

In [18]:
# To make the output easier to read, let's also disable the output from Gurobi:
m.Params.outputflag = 0

results_revenue = np.zeros(nSimulations)

for s in range(nSimulations):
    total_revenue = 0.0
    b = B.copy()
    
    for t in range(T):
        
        i = int(arrival_sequences[s, t])
        
        if (i < n):
            dual_val = bpc(b,t)
            
            days = [day for day in range(7) if day+1>=hotel.iloc[i,1] and day+1<hotel.iloc[i,2]]
            
            # Compute the total bid price of the request:
            total_bid_price = sum([dual_val[day][req_to_rooms[i]] for day in days])
            
            if ( (revenue[i] >= total_bid_price) and (b[days, req_to_rooms[i]] > 0).all() ):
                b[days, req_to_rooms[i]] -= 1
                total_revenue += revenue[i]
    
    results_revenue[s] = total_revenue
    

mean_LP_revenue = results_revenue.mean()

print(f"The average revenue of this policy over the 100 random request sequences is ${mean_LP_revenue}.")

The average revenue of this policy over the 100 random request sequences is $163679.075.


In [19]:
# Sorting indices based on decreasing order of probability
prob_index = np.flipud(np.argsort(probability))
print(prob_index)
probability[prob_index]

[13 41  8 35 46 22 69  9 36 64  0 14 28 47 50 19  1  2 75  7 63 78 52 16
 48 15 51 57 56 58 11 10  4  3 24 23 25 26 27 21 20 72 18 17 30 79 12 80
 81  6  5 29 76 31 53 70 73 68 67 66 65 74 62 61 60 59 55 54 49 32 45 44
 43 42 82 40 39 38 37 71 77 34 33 83]


array([0.07407407, 0.03703704, 0.03703704, 0.03703704, 0.02962963,
       0.02222222, 0.01851852, 0.01851852, 0.01851852, 0.01851852,
       0.01851852, 0.01481482, 0.01481482, 0.01481482, 0.01481482,
       0.01481482, 0.01111111, 0.01111111, 0.01111111, 0.01111111,
       0.01111111, 0.01111111, 0.00740741, 0.00740741, 0.00740741,
       0.00740741, 0.00740741, 0.00740741, 0.00740741, 0.00740741,
       0.00740741, 0.00740741, 0.00740741, 0.00740741, 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 ,
       0.0037037 , 0.0037037 , 0.0037037 , 0.0037037 , 0.00370

In [20]:
# Getting the revenue correspondinfg to the sorted indices
revenue[prob_index]

array([ 200. ,  250. ,  400. ,  250. ,  287.5,  230. ,  300. ,  630. ,
        500. ,  600. ,  200. ,  430. ,  250. ,  575. ,  287.5,  460. ,
        400. ,  600. ,  690. ,  200. ,  300. ,  345. ,  825. ,  890. ,
        862.5,  660. ,  575. ,  600. ,  300. ,  900. , 1090. ,  860. ,
       1060. ,  830. ,  660. ,  460. ,  230. ,  430. ,  200. ,  890. ,
        690. , 1335. ,  230. , 1090. ,  750. ,  690. , 1290. ,  990. ,
        345. , 1490. , 1290. ,  500. , 1035. , 1037.5,  287.5,  645. ,
       1635. , 1935. , 1635. , 1290. ,  945. ,  345. , 2235. , 1935. ,
       1590. , 1245. ,  250. ,  537.5, 1112.5, 1325. , 1362.5, 1112.5,
        825. ,  537.5,  645. , 1612.5, 1362.5, 1075. ,  787.5,  990. ,
       1335. , 1862.5, 1612.5,  300. ])